In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By

import pandas as pd
from simpledbf import Dbf5

<frozen importlib._bootstrap>:228: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


PyTables is not installed. No support for HDF output.


In [5]:
mapa_uf = (
(12, ('Acre', 'AC')),
(27, ('Alagoas', 'AL')),
(16, ('Amapá', 'AP')),
(13, ('Amazonas', 'AM')),
(29, ('Bahia', 'BA')),
(23, ('Ceará', 'CE')),
(53, ('Distrito Federal', 'DF')),
(32, ('Espírito Santo', 'ES')),
(52, ('Goiás', 'GO')),
(21, ('Maranhão', 'MA')),
(51, ('Mato Grosso', 'MT')),
(50, ('Mato Grosso do Sul', 'MS')),
(31, ('Minas Gerais', 'MG')),
(15, ('Pará', 'PA')),
(25, ('Paraíba', 'PB')),
(41, ('Paraná', 'PR')),
(26, ('Pernambuco', 'PE')),
(22, ('Piauí', 'PI')),
(24, ('Rio Grande do Norte', 'RN')),
(43, ('Rio Grande do Sul', 'RS')),
(33, ('Rio de Janeiro', 'RJ')),
(11, ('Rondônia', 'RO')),
(14, ('Roraima', 'RR')),
(42, ('Santa Catarina', 'SC')),
(35, ('São Paulo', 'SP')),
(28, ('Sergipe', 'SE')),
(17, ('Tocantins', 'TO')),
)

In [6]:
import sqlite3
from datetime import datetime, date
# Create your connection.
cnx = sqlite3.connect('db.sqlite')


In [ ]:
dbf = Dbf5('./DENGBRDBF/DENGBR15.dbf')
df = dbf.to_dataframe()

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")

driver = webdriver.Chrome(chrome_options=options)
# driver = webdriver.Chrome()
driver.get("C:\workspace\ic ia\municipios.html");
# html_options = municipios.find_elements_by_css_selector('option')
html_options = driver.find_elements(By.CSS_SELECTOR, "option")

In [67]:
dict_munic = {'codigo': [], 'municipio': []}
for element in html_options:
    texto = element.text.split(' ')
    dict_munic['codigo'].append(texto[0])
    dict_munic['municipio'].append(''.join(texto[1:]))

df_munic = pd.DataFrame(dict_munic)
df_munic = df_munic.iloc[1:]
df_munic

,codigo,municipio
1,MUNICIPIO,IGNORADO-RO
2,110001,ALTAFLORESTAD'OESTE
3,110002,ARIQUEMES
4,110003,CABIXI
5,110004,CACOAL
...,...,...
5593,522205,VICENTINOPOLIS
5594,522220,VILABOA
5595,522230,VILAPROPICIO
5596,MUNICIPIO,IGNORADO-DF


In [127]:
from copy import deepcopy
df_munic_with_uf = deepcopy(df_munic)
ranges = list(df_munic[df_munic['codigo'] == 'MUNICIPIO'].index)
list_of_ranges = [range(ranges[i], ranges[i+1]-1) for i in range(len(ranges) - 1)]
list_of_ranges.append(range(ranges[-1], len(df_munic)))
# print(df_munic.iloc[ranges[0]-1]['municipio'].replace('IGNORADO-', ''))
# len(ranges), len(list_of_ranges)
# list_of_ranges
# df_munic_with_uf['uf'] = df_munic.iloc[:]
lista_ufs = []
[x for x in df_munic.loc[:].index]
for num_row in range(1, len(df_munic) + 1):
    for i, rang in enumerate(list_of_ranges):
        if num_row in rang:
            lista_ufs.append(df_munic.loc[ranges[i]]['municipio'].replace('IGNORADO-', ''))
        if num_row == ranges[i]:
            lista_ufs.append('-')
for i, uf in enumerate(lista_ufs):
    if uf == '-':
        lista_ufs[i] = lista_ufs[i-1]
        lista_ufs[i-1] = '-'
df_munic_with_uf['UF'] = lista_ufs
df_munic_with_uf = df_munic_with_uf[df_munic_with_uf['UF'] != '-'].reset_index(drop=True)


In [130]:
df_munic_with_uf.to_sql(name='munic_uf', con=cnx)

5570

In [68]:
range(100, 201)

range(100, 201)

In [ ]:
df_munic.to_sql(name='datasus_municipios', con=cnx)

In [ ]:
pd.read_sql('select * from price2', cnx)

In [ ]:
price2 = pd.DataFrame({'col1': [9, 59,9.71], 'col2': [11,32,93],'col3': [date(2022, 12,1), date(2022,6,12), date(2022,12,17)]})
price2.to_sql(name='price2', con=cnx, if_exists='append')

In [33]:
dbf = Dbf5('./Bases/DENGBRDBF/DENGBR00.dbf')
df2 = dbf.to_dataframe()

In [35]:
a = df2.columns

In [37]:
[x for x in a if 'DT' in x]

['DT_NOTIFIC',
 'DT_SIN_PRI',
 'ANT_DT_INV',
 'DT_DOSE',
 'DT_FEBRE',
 'DT_CHOQUE',
 'DT_INTERNA',
 'DT_COL_HEM',
 'DT_COL_PLQ',
 'DT_COL_HE2',
 'DT_COL_PL2',
 'DT_SORO1',
 'DT_SORO2',
 'DT_SOROR1',
 'DT_SOROR2',
 'CON_DT_OBI',
 'CON_DT_ENC']

In [43]:
df2[df2['CON_DT_OBI'].notna()]
df2[df2['CON_DT_OBI'].notna()][['CON_DT_OBI', 'DT_NOTIFIC']]

,CON_DT_OBI,DT_NOTIFIC
46338,2000-06-13,2000-05-29
47672,2000-01-12,2000-01-11
100735,2000-05-14,2000-05-12
158794,2000-04-15,2000-04-15
169502,2000-04-27,2000-04-18


In [155]:
dbf = Dbf5('./Bases/DENGBRDBF/DENGBR01.dbf', codec='latin-1')
df5 = dbf.to_dataframe()

In [52]:
df3.index += 12313

In [53]:
df3

,TP_NOT,ID_AGRAVO,DT_NOTIFIC,SEM_NOT,NU_ANO,SG_UF_NOT,ID_MUNICIP,ID_REGIONA,ID_UNIDADE,DT_SIN_PRI,...,PLASMATICO,EVIDENCIA,PLAQ_MENOR,CON_FHD,COMPLICA,HOSPITALIZ,DT_INTERNA,UF,MUNICIPIO,TP_SISTEMA
12313,2,A90,2009-01-19,200903,2009,33,330340,NaN,2271877,2009-01-07,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1
12314,2,A90,2009-01-28,200904,2009,21,210780,1577,2454726,2009-01-24,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1
12315,2,A90,2009-01-28,200904,2009,21,210780,1577,2454726,2009-01-26,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1
12316,2,A90,2009-01-12,200902,2009,29,292150,1410,3340392,2009-01-06,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1
12317,2,A90,2009-01-29,200904,2009,29,292150,1410,3340392,2009-01-19,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612966,2,A90,2009-12-23,200951,2009,52,520430,1799,NaN,2009-12-20,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1
612967,2,A90,2009-12-07,200949,2009,52,520430,1799,NaN,2009-11-27,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1
612968,2,A90,2009-12-07,200949,2009,52,520430,1799,NaN,2009-11-27,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1
612969,2,A90,2009-12-29,200952,2009,52,520130,1779,NaN,2009-12-11,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1


In [65]:
[x for x in df3.columns if 'DT' in x]
[x for x in df3.columns if 'ANO' in x]
[x for x in df3.columns if 'MUNI' in x]
[x for x in df3.columns if 'GRAV' in x]
# True if not df3.get('CON_DT_OBI') else False
df3[df3['ID_AGRAVO'].notna()]

,TP_NOT,ID_AGRAVO,DT_NOTIFIC,SEM_NOT,NU_ANO,SG_UF_NOT,ID_MUNICIP,ID_REGIONA,ID_UNIDADE,DT_SIN_PRI,...,PLASMATICO,EVIDENCIA,PLAQ_MENOR,CON_FHD,COMPLICA,HOSPITALIZ,DT_INTERNA,UF,MUNICIPIO,TP_SISTEMA
12313,2,A90,2009-01-19,200903,2009,33,330340,NaN,2271877,2009-01-07,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1
12314,2,A90,2009-01-28,200904,2009,21,210780,1577,2454726,2009-01-24,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1
12315,2,A90,2009-01-28,200904,2009,21,210780,1577,2454726,2009-01-26,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1
12316,2,A90,2009-01-12,200902,2009,29,292150,1410,3340392,2009-01-06,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1
12317,2,A90,2009-01-29,200904,2009,29,292150,1410,3340392,2009-01-19,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612966,2,A90,2009-12-23,200951,2009,52,520430,1799,NaN,2009-12-20,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1
612967,2,A90,2009-12-07,200949,2009,52,520430,1799,NaN,2009-11-27,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1
612968,2,A90,2009-12-07,200949,2009,52,520430,1799,NaN,2009-11-27,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1
612969,2,A90,2009-12-29,200952,2009,52,520130,1779,NaN,2009-12-11,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,1


In [134]:
df_munic_with_uf[df_munic_with_uf['codigo'] == '120017'].iloc[0]['UF']

'AC'

In [157]:
index = 0
for i in range(7, 22):
    print(f'lendo DENGBR{i:02}.dbf...')
    dbf = Dbf5(f'./Bases/DENGBRDBF/DENGBR{i:02}.dbf', codec='latin-1')
    df_dados = dbf.to_dataframe()
    colunas = ['DT_NOTIFIC', 'NU_ANO', 'ID_MUNICIP', 'SG_UF', 'DT_OBITO']
    print(f'inserindo DENGBR{i:02}.dbf...')
    # if not df_dados.get('DT_OBITO'):
    #     df_dados = df_dados.rename({'CON_DT_OBI': 'DT_OBITO'}, axis=1)
    
    df_dados = df_dados[colunas]
    # df_dados['UF'] = df_dados['ID_MUNICIP'].apply(lambda cod_mun: df_munic_with_uf[df_munic_with_uf['codigo'] == cod_mun].iloc[0]['UF'])

    df_dados.index += index
    
    df_dados.to_sql(name='casos_dengue', con=cnx, if_exists='append')
        
    index += len(df_dados)

lendo DENGBR07.dbf...
inserindo DENGBR07.dbf...
lendo DENGBR08.dbf...
inserindo DENGBR08.dbf...
lendo DENGBR09.dbf...
inserindo DENGBR09.dbf...
lendo DENGBR10.dbf...
inserindo DENGBR10.dbf...
lendo DENGBR11.dbf...
inserindo DENGBR11.dbf...
lendo DENGBR12.dbf...
inserindo DENGBR12.dbf...
lendo DENGBR13.dbf...
inserindo DENGBR13.dbf...
lendo DENGBR14.dbf...
inserindo DENGBR14.dbf...
lendo DENGBR15.dbf...
inserindo DENGBR15.dbf...
lendo DENGBR16.dbf...
inserindo DENGBR16.dbf...
lendo DENGBR17.dbf...
inserindo DENGBR17.dbf...
lendo DENGBR18.dbf...
inserindo DENGBR18.dbf...
lendo DENGBR19.dbf...
inserindo DENGBR19.dbf...
lendo DENGBR20.dbf...
inserindo DENGBR20.dbf...
lendo DENGBR21.dbf...
inserindo DENGBR21.dbf...


In [151]:
df_dados

df_munic_with_uf[df_munic_with_uf['codigo'] == '']
df_dados['ID_MUNICIP'].iloc[0]
# df_munic_with_uf['codigo'].iloc[0]
#'120385'
# df_munic_with_uf[df_munic_with_uf['codigo'].str.startswith('120')]
df_dados['ID_MUNICIP']

0         1200385
1         1200385
2         1200385
3         1200385
4         1200500
           ...   
172850    1713205
172851    1713205
172852    1713205
172853    1713205
172854    1713205
Name: ID_MUNICIP, Length: 172855, dtype: object

In [7]:
df_dengue = pd.read_sql('select * from casos_dengue', cnx)

In [165]:
df_dengue2 = df_dengue.iloc[5000000:5100000]

In [185]:
# df_dengue2[df_dengue2['DT_OBITO'].notna()]
# datas = df_dengue2['DT_NOTIFIC'].unique()[0]
df_casos = pd.DataFrame({'datas': pd.date_range(date(2000, 1, 1), date(2021,12,31)), 
              'casos': [len(df_dengue[df_dengue['DT_NOTIFIC'] == dt.strftime('%Y-%m-%d')]) 
                        for dt in pd.date_range(date(2000, 1, 1), date(2021,12,31))]
              })

In [338]:
df_obitos = pd.DataFrame({'datas': pd.date_range(date(2000, 1, 1), date(2021,12,31)), 
              'casos': [len(df_dengue[df_dengue['DT_OBITO'] == dt.strftime('%Y-%m-%d')]) 
                        for dt in pd.date_range(date(2000, 1, 1), date(2021,12,31))]
              })

In [200]:
meses = pd.date_range(date(2000, 1, 1), date(2021,12,31), freq='MS')

In [236]:
import calendar
from datetime import datetime, timedelta
dt = datetime(2022, 9, 30)
# print("The original date is:", input_dt.date())
def get_next_month(dt):
    return (date(dt.year, dt.month, calendar.monthrange(dt.year, dt.month)[1])+ timedelta(days=1)).strftime('%Y-%m-%d')
# print(f"Last date of month is: {input_dt.year}-{input_dt.month}-{day}")
get_next_month(dt)

'2022-10-01'

In [223]:
# meses = pd.date_range(date(2000, 1, 1), date(2021,12,31))
meses = pd.date_range(date(2010, 1, 1), date(2010,5,31), freq='MS')
# df_uf_mensal = pd.DataFrame({'datas': meses,
#                              'casos': [len(df_dengue[(df_dengue['DT_NOTIFIC'] < meses[i+1].strftime('%Y-%m-%d')) & (df_dengue['DT_NOTIFIC'] > meses[i].strftime('%Y-%m-%d'))]) 
#                                         for i, dt in enumerate(meses) if i != len(meses) - 1]
#                             })
df_uf_mensal = pd.DataFrame({'datas':[], 'casos':[], 'uf':[]})
ufs = df_dengue2['SG_UF'].unique()

# casos = [len(df_dengue[(df_dengue['DT_NOTIFIC'] < meses[i+1].strftime('%Y-%m-%d')) & (df_dengue['DT_NOTIFIC'] > meses[i].strftime('%Y-%m-%d'))]) 
#                                         for i, dt in enumerate(meses) if i != len(meses) - 1]
# datas = meses
ufs

array(['50', '11', '51', '52', '41', '35', '53', '43', '31', '42', '13',
       None, '15', '12', '33', '26', '17', '27', '23', '24', '22', '29',
       '14', '32', '28', '16', '21', '25'], dtype=object)

In [238]:
get_next_month(pd.date_range(date(2010, 1, 1), date(2010,5,31), freq='MS')[0])

'2010-02-01'

In [264]:
df_agg = df_dengue.groupby('SG_UF').agg(lambda x: list(set(x))).reset_index()

In [275]:
valid_codes = [str(cod) for cod, tupla in mapa_uf]
valid_acron = [str(acronimo_estado) for cod, (nome_estado, acronimo_estado) in mapa_uf]
df_agg[~df_agg['SG_UF'].isin(valid_codes)]
df_agg[df_agg['SG_UF'].isin(valid_codes)]
# valid_codes
valid_acron

['AC',
 'AL',
 'AP',
 'AM',
 'BA',
 'CE',
 'DF',
 'ES',
 'GO',
 'MA',
 'MT',
 'MS',
 'MG',
 'PA',
 'PB',
 'PR',
 'PE',
 'PI',
 'RN',
 'RS',
 'RJ',
 'RO',
 'RR',
 'SC',
 'SP',
 'SE',
 'TO']

In [278]:
import numpy as np
from copy import deepcopy
valid_codes = [str(cod) for cod, tupla in mapa_uf]
valid_acron = [str(acronimo_estado) for cod, (nome_estado, acronimo_estado) in mapa_uf]
df_dengue_acronimos_tratados = deepcopy(df_dengue)
df_dengue_acronimos_tratados['SG_UF'] = df_dengue_acronimos_tratados['SG_UF'].apply(lambda x: 
                                                                                    x if x in valid_acron    # Se for o acrônimo válido
                                                                                    else ([acronimo for cod, (_, acronimo) in mapa_uf if str(x).strip() == str(cod).strip()][0] if x in valid_codes  # Se tiver um código válido
                                                                                    else np.nan))  # Se não for nenhum dos dois

In [316]:
df_agg = df_dengue_acronimos_tratados.groupby('SG_UF').agg(lambda x: list(set(x))).reset_index()

In [324]:
total_sum = 0
for i, row in df_agg.iterrows():
    total_sum += len(row['DT_NOTIFIC'])
total_sum

198816

In [327]:
df_dengue_acronimos_tratados[df_dengue_acronimos_tratados['SG_UF'].isin(valid_acron)]

,index,DT_NOTIFIC,NU_ANO,ID_MUNICIP,SG_UF,DT_OBITO
0,0,2000-12-09,2000,1200385,AC,None
1,1,2000-04-14,2000,1200385,AC,None
2,2,2000-04-10,2000,1200385,AC,None
3,3,2000-09-21,2000,1200385,AC,None
4,4,2000-12-04,2000,1200500,AC,None
...,...,...,...,...,...,...
21964989,19180452,2021-12-27,2021,171090,TO,None
21964990,19180453,2021-12-03,2021,171050,TO,None
21964991,19180454,2021-12-28,2021,171090,TO,None
21964992,19180455,2021-12-08,2021,170600,TO,None


In [333]:
# meses = [dt.strftime('%Y-%m-%d') for dt in pd.date_range(date(2010, 1, 1), date(2010,5,31), freq='MS')]

df_estados = None
import numpy as np
meses = pd.date_range(date(2000, 1, 1), date(2021,12,31), freq='MS')
valid_codes = [str(cod) for cod, tupla in mapa_uf]
valid_acron = [str(acronimo_estado) for cod, (nome_estado, acronimo_estado) in mapa_uf]



# for index, estado in df_dengue_acronimos_tratados.groupby('SG_UF').agg(lambda x: list(set(x))).reset_index().iterrows():
# for index, estado in df_agg:
for estado in valid_acron:
    # print(estado)
    quantidade_por_mes = []
    df_est = df_dengue_acronimos_tratados[df_dengue_acronimos_tratados['SG_UF'] == estado]
    for mes in meses:
        mes_str = mes.strftime('%Y-%m-%d')
        # print(mes_str[2:4]+mes_str[4:7], end=' ')
        next_mes_str = get_next_month(mes)
        quantidade_casos = 0
        for dt_caso in df_est['DT_NOTIFIC']:
            if dt_caso and dt_caso < next_mes_str and dt_caso > mes_str: 
                quantidade_casos += 1
                # print(dt_caso)
        quantidade_por_mes.append(quantidade_casos)
        # break
    # break
    # uf_acronimo = np.nan
    # for cod, (nome_estado, acronimo_estado) in mapa_uf:
    #     if estado['SG_UF'] in valid_codes and str(cod) == str(estado['SG_UF']).strip(): uf_acronimo = acronimo_estado
    #     elif estado['SG_UF'] in valid_acron and str(acronimo_estado) == str(estado['SG_UF']).strip(): uf_acronimo = acronimo_estado
    # print(uf_acronimo)
    uf_acronimo = estado
    # print('Montando estado')
    df_estado = pd.DataFrame({'UF': [uf_acronimo for i in range(len(meses))], 
                               'CASOS': quantidade_por_mes, 
                               'MES': [dt.strftime('%Y-%m-%d') for dt in meses]})
    
    df_estados = pd.concat([df_estados, df_estado]).reset_index(drop=True) if not df_estados is None else df_estado
    # break
df_estados

,UF,CASOS,MES
0,AC,9,2000-01-01
1,AC,18,2000-02-01
2,AC,29,2000-03-01
3,AC,34,2000-04-01
4,AC,65,2000-05-01
...,...,...,...
7123,TO,666,2021-08-01
7124,TO,525,2021-09-01
7125,TO,625,2021-10-01
7126,TO,2168,2021-11-01


In [308]:
# df_estados[(df_estados['UF'] == 'SP') & (df_estados['MES'] == '2015-03-01')]
sum([int(row['CASOS']) for i, row in df_estados.iterrows()])

192375

In [239]:
['uf_acronimo' for i in range(len(meses))]

['uf_acronimo', 'uf_acronimo', 'uf_acronimo', 'uf_acronimo', 'uf_acronimo']

In [208]:
datas, casos, [dt for i, dt in enumerate(meses) if i != len(meses) - 1]

(DatetimeIndex(['2010-01-01', '2010-02-01', '2010-03-01', '2010-04-01',
                '2010-05-01'],
               dtype='datetime64[ns]', freq='MS'),
 [118880, 175694, 278925, 246553],
 [Timestamp('2010-01-01 00:00:00', freq='MS'),
  Timestamp('2010-02-01 00:00:00', freq='MS'),
  Timestamp('2010-03-01 00:00:00', freq='MS'),
  Timestamp('2010-04-01 00:00:00', freq='MS')])

In [202]:
[meses[i+1] for i, dt in enumerate(meses) if i != len(meses) - 1]

Timestamp('2021-12-01 00:00:00', freq='MS')

In [193]:
from datetime import timedelta
import dateutil.relativedelta as relativedelta

pd.date_range(date(2000, 1, 1), date(2021,12,31), freq='MS')

DatetimeIndex(['2000-01-01', '2000-02-01', '2000-03-01', '2000-04-01',
               '2000-05-01', '2000-06-01', '2000-07-01', '2000-08-01',
               '2000-09-01', '2000-10-01',
               ...
               '2021-03-01', '2021-04-01', '2021-05-01', '2021-06-01',
               '2021-07-01', '2021-08-01', '2021-09-01', '2021-10-01',
               '2021-11-01', '2021-12-01'],
              dtype='datetime64[ns]', length=264, freq='MS')

In [187]:
df_casos.to_sql(name='num_casos_dengue', con=cnx, if_exists='append')

8036

In [188]:
df_casos

,datas,casos
0,2000-01-01,13
1,2000-01-02,97
2,2000-01-03,341
3,2000-01-04,386
4,2000-01-05,422
...,...,...
8031,2021-12-27,3337
8032,2021-12-28,3151
8033,2021-12-29,2908
8034,2021-12-30,2236


In [337]:
df_estados.to_sql(name='uf_casos_dengue', con=cnx, if_exists='append')

7128

In [341]:
df_obitos.to_sql(name='num_obitos_dengue', con=cnx, if_exists='append')

8036

In [8]:
df_obitos = pd.read_sql('select * from num_obitos_dengue', cnx)

In [336]:
sum(df_estados['CASOS'])

21318571

In [180]:
dt = date(2010, 1, 15)
df_dengue2[df_dengue2['DT_NOTIFIC'] == dt.strftime('%Y-%m-%d')]

,index,DT_NOTIFIC,NU_ANO,ID_MUNICIP,SG_UF,DT_OBITO
5021642,2237105,2010-01-15,2010,310820,31,None
5021692,2237155,2010-01-15,2010,110018,11,None
5021734,2237197,2010-01-15,2010,110018,11,None
5021735,2237198,2010-01-15,2010,110018,11,None
5021786,2237249,2010-01-15,2010,313420,31,None
...,...,...,...,...,...,...
5065893,2281356,2010-01-15,2010,290850,29,None
5065998,2281461,2010-01-15,2010,310620,31,None
5066021,2281484,2010-01-15,2010,110020,11,None
5066022,2281485,2010-01-15,2010,110020,11,None


In [348]:
len(pd.date_range(date(2023, 3, 12), date(2023,11,23), freq='W'))//4

9

In [2]:
import pandas as pd
import sqlite3
from datetime import datetime, date
cnx = sqlite3.connect('db.sqlite')

df_obitos = pd.read_sql('select * from num_obitos_dengue', cnx)
df_casos = pd.read_sql('select * from num_casos_dengue', cnx)
df_dengue = pd.read_sql('select * from casos_dengue', cnx)
df_estado = pd.read_sql('select * from uf_casos_dengue', cnx)


In [5]:
df_casos

,index,datas,casos
0,0,2000-01-01 00:00:00,13
1,1,2000-01-02 00:00:00,97
2,2,2000-01-03 00:00:00,341
3,3,2000-01-04 00:00:00,386
4,4,2000-01-05 00:00:00,422
...,...,...,...
8031,8031,2021-12-27 00:00:00,3337
8032,8032,2021-12-28 00:00:00,3151
8033,8033,2021-12-29 00:00:00,2908
8034,8034,2021-12-30 00:00:00,2236
